In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pypsdm.io.utils import get_absolute_path_from_project_root, get_absolute_path_from_working_dir
from pypsdm.models.input.container.grid import GridContainer


grid_path = "./simple_raw_grid"
sep = ","
grid = GridContainer.from_csv(
    grid_path,
    sep
)

In [3]:
target_grid_path = "./simple_grid"

In [4]:
from pypsdm.plots.grid import grid_plot


grid_plot(grid)

In [5]:
node_0 = "df97c0d1-379b-417a-a473-8e7fe37da99d"
node_1 = "6a4547a8-630b-46e4-8144-9cd649e67c07"
node_2 = "b7a5be0d-2662-41b2-99c6-3b8121a75e9e"
node_3 = "1dcddd06-f41a-405b-9686-7f7942852196"
node_4 = "e3c3c6a3-c383-4dbb-9b3f-a14125615386"

In [6]:
from uuid import uuid4
from pypsdm.models.input.create.participants import create_pvs

from pypsdm.models.input.participant.pv import PhotovoltaicPowerPlants

pv_s_rated = [4, 6, 10]

data_dict = {
    "id": ["PV_NS_02", "PV_NS_03", "PV_NS_04"],
	"node": [node_2, node_3, node_4],
    "s_rated": pv_s_rated,
	"azimuth": [90, 0, -90],
	"elevation_angle": [40 , 45, 30]
}

pvs = create_pvs(data_dict)

In [7]:
from re import I
from pypsdm.models.input.create.participants import create_storages


data_dict = {
    "id": ["BS_NS_02", "BS_NS_03", "BS_NS_04"],
    "node": [node_2, node_3, node_4],
    "e_storage": [1.5 * pv_power for pv_power in pv_s_rated],
}

bs = create_storages(data_dict)

In [8]:
from pypsdm.models.input.create.participants import create_electric_vehicles
from pypsdm.models.input.participant.evs import ElectricVehicles


ev_s_rated = [10, 10, 20]
data_dict = {
	"id": ["EV_NS_02", "EV_NS_03", "EV_NS_04"],
	"node": [node_2, node_3, node_4],
	"s_rated": ev_s_rated,
	"e_storage": [80, 70, 100],
	"e_cons": [20, 16, 19],
}

evs = create_electric_vehicles(data_dict)

In [9]:
from pipes import Template
from pypsdm.models.enums import ElectricCurrentType
from pypsdm.models.input.create.participants import create_ev_charging_stations
from pypsdm.models.input.participant.evcs import EvChargingStations, EvcsLocationType
from string import Template

data_dict = {
    "id": ["EVCS_NS_02", "EVCS_NS_03", "EVCS_NS_04"],
    "node": [node_2, node_3, node_4],
    "location_type": [EvcsLocationType.HOME.value, EvcsLocationType.HOME.value, EvcsLocationType.HOME.value],
	"s_rated": [10, 10, 10],
}

evcs = create_ev_charging_stations(data_dict)

/var/folders/bt/fjsy8b510jscjpws11www50c0000gn/T/ipykernel_28179/2700485612.py:1: DeprecationWarning:

'pipes' is deprecated and slated for removal in Python 3.13



In [10]:
evs.filter_by_nodes(node_2).uuid

a3e4709e-1434-4723-83db-80468c85b31a    a3e4709e-1434-4723-83db-80468c85b31a
dtype: object

In [11]:
from pypsdm.models.input.create.thermal import create_thermal_busses


data_dict = {
    "id": ["TB_NS_02", "TB_NS_03", "TB_NS_04"],
}

tbs = create_thermal_busses(data_dict)

In [12]:
from pypsdm.models.input.create.participants import create_heat_pumps


data_dict = {
    "id": ["HP_NS_02", "HP_NS_03", "HP_NS_04"],
    "node": [node_2, node_3, node_4],
    "thermal_bus": list(tbs.uuid),
    "s_rated": [4, 3, 6],
}

hps = create_heat_pumps(data_dict)

In [13]:
from pypsdm.models.input.create.thermal import create_thermal_houses


data_dict = {
    "thermal_bus": list(tbs.uuid),
    "id": ["TH_NS_02", "TH_NS_03", "TH_NS_04"],
    "eth_losses": [0.101, 0.121, 0.131],
    "eth_capa": [8.801, 9.121, 12.131],
}

ths = create_thermal_houses(data_dict)

In [14]:
from pypsdm.models.input.create.participants import create_loads


load_data_dict = {
	"id": ["L_NS_02", "L_NS_03", "L_NS_04"],
	"node": [node_2, node_3, node_4],
    "s_rated": [3, 3, 3],
	"e_cons_annual": [3000, 3500, 4000],
	"load_profile": ["h0", "h0", "h0"]
}

loads = create_loads(load_data_dict)

In [15]:
from pypsdm.models.input.container.participants import SystemParticipantsContainer
from pypsdm.models.input.participant.bm import BiomassPlants
from pypsdm.models.input.participant.em import EnergyManagementSystems
from pypsdm.models.input.participant.fixed_feed_in import FixedFeedIns
from pypsdm.models.input.participant.load import Loads
from pypsdm.models.input.participant.wec import WindEnergyConverters


participants = SystemParticipantsContainer(
    ems = EnergyManagementSystems.create_empty(),
    loads = loads,
    pvs = pvs,
    storages = bs,
    evs = evs,
    evcs = evcs,
    hps = hps, 
    wecs = WindEnergyConverters.create_empty(),
    fixed_feed_ins= FixedFeedIns.create_empty(),
    biomass_plants=BiomassPlants.create_empty(),
)

In [16]:
node_participants = participants.build_node_participants_map(grid.raw_grid.nodes)

In [17]:
node_connected_assets = {}
for node, nodal_participants in node_participants.items():
    participant_uuids = []
    for current in nodal_participants.to_list():
        if isinstance(current, ElectricVehicles):
            continue
        participant_uuids.extend(list(current.uuid))
    node_connected_assets[node] = participant_uuids

In [18]:

from pypsdm.models.input.create.participants import create_energy_management_systems


data_dict = {
    "id": ["EMS_NS_02", "EMS_NS_03", "EMS_NS_04"],
    "node": [node_2, node_3, node_4],
	"connected_assets": [node_connected_assets[node] for node in [node_2, node_3, node_4]],
	"control_strategy": ["self_optimization", "self_optimization", "self_optimization"]
}

emss = create_energy_management_systems(data_dict)

In [19]:
updated_participants = SystemParticipantsContainer(
    emss,
    participants.loads,
    participants.fixed_feed_ins,
    participants.pvs,
    participants.biomass_plants,
    participants.wecs,
    participants.storages,
    participants.evs,
    participants.evcs,
    participants.hps,
)

In [20]:
node_participants_map = updated_participants.build_node_participants_map(grid.raw_grid.nodes)
updated_grid = GridContainer(
    grid.raw_grid,
    updated_participants,
    grid.primary_data,
    node_participants_map
)

In [21]:
from pypsdm.models.input.thermal.grid import ThermalGridContainer


tg = ThermalGridContainer(
    tbs,
    ths
)

In [22]:
updated_grid.to_csv(target_grid_path, include_primary_data=False)
tg.to_csv(target_grid_path)

# EV Mobility Points of Interest

In [23]:
import pandas as pd

public_pois = pd.read_csv("./resources/public_pois.csv", index_col="uuid")

In [31]:
public_pois

,id,size,lat,lon,categoricallocation
uuid,,,,,
2da545dc-a649-4500-9f1f-0f35baec306d,WORK-poi,100.0,51.484328,7.411648,work
3795354b-8eed-4a43-9aaa-9d0df805c168,SUPERMARKET-poi,100.0,51.484328,7.411648,supermarket
6537e3bf-5da3-4846-aaae-e8e9fec2516a,SERVICES-poi,100.0,51.484328,7.411648,services
c970bce7-a542-4388-bc36-1cd2c9e6b0cd,OTHER_SHOP-poi,100.0,51.484328,7.411648,other_shop
198a66cf-a342-471c-ae1d-54e92f3390f0,MEDICINAL-poi,100.0,51.484328,7.411648,medicinal
64e7dccd-9ceb-45ee-9382-ac04e7006713,BBPG-poi,100.0,51.484328,7.411648,bbpg
5aeb7096-d6e3-4219-aa40-f44f1bebd279,RESTAURANT-poi,100.0,51.484328,7.411648,restaurant
257c8e57-9b0a-4ea0-a95f-7f07e2939c9a,CULTURE-poi,100.0,51.484328,7.411648,culture
699bec33-58cc-4c28-baf4-5b0ded6e8e65,SPORTS-poi,100.0,51.484328,7.411648,sports


In [41]:
from pypsdm.models.input.create.poi import create_poi_mapping


pois, poi_mappings = create_poi_mapping(updated_grid, public_pois)

poi_mappings

,poi,evcs,evs
0,a71a760e-73aa-4a46-8a89-e5f129b236de,93a199f7-7a82-46f2-a853-e388885a0269,a3e4709e-1434-4723-83db-80468c85b31a
0,ee43547a-ff1f-436f-bbe3-413dbce1d7b3,b7394b96-ce8d-44b2-bd90-8451be462aee,8e59850b-3389-4774-bad3-1cdce5fc7ce4
0,2c12d39e-a9a4-44c8-9cab-ea05f66be6ae,4ec3962c-efd7-4b08-8504-c8f78d73d120,9223dade-054a-4f97-826a-34ef7dd0ddfb


In [40]:
pois 

,id,size,lat,lon,categoricallocation,id,lat,lon,categoricallocation,size,id,lat,lon,categoricallocation,size,id,lat,lon,categoricallocation,size
2da545dc-a649-4500-9f1f-0f35baec306d,WORK-poi,100.0,51.484328,7.411648,work,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3795354b-8eed-4a43-9aaa-9d0df805c168,SUPERMARKET-poi,100.0,51.484328,7.411648,supermarket,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6537e3bf-5da3-4846-aaae-e8e9fec2516a,SERVICES-poi,100.0,51.484328,7.411648,services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
c970bce7-a542-4388-bc36-1cd2c9e6b0cd,OTHER_SHOP-poi,100.0,51.484328,7.411648,other_shop,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198a66cf-a342-471c-ae1d-54e92f3390f0,MEDICINAL-poi,100.0,51.484328,7.411648,medicinal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64e7dccd-9ceb-45ee-9382-ac04e7006713,BBPG-poi,100.0,51.484328,7.411648,bbpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5aeb7096-d6e3-4219-aa40-f44f1bebd279,RESTAURANT-poi,100.0,51.484328,7.411648,restaurant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
257c8e57-9b0a-4ea0-a95f-7f07e2939c9a,CULTURE-poi,100.0,51.484328,7.411648,culture,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
699bec33-58cc-4c28-baf4-5b0ded6e8e65,SPORTS-poi,100.0,51.484328,7.411648,sports,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21bedd21-6a82-416f-a19c-55cd1a8de461,RELIGIOUS-poi,100.0,51.484328,7.411648,religious,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
poi_mappings["poi"]

0    id                      HOME-poi
lat          ...
0    id                      HOME-poi
lat          ...
0    id                      HOME-poi
lat          ...
Name: poi, dtype: object

In [ ]:
import os


pois_path = os.path.join(target_grid_path, "pois")
if not os.path.exists(pois_path):
    os.mkdir(pois_path)
pois.to_csv(pois_path + "/poi.csv", index_label="uuid")
poi_mappings.to_csv(pois_path + "/pois_mapping.csv", index=False)